In [1]:
import os
from pathlib import Path
from dotenv import load_dotenv

import json
import datetime

from openai import AzureOpenAI
from pydantic import BaseModel, Field

from prompts.StateDiagram import SYSTEM_PROMPT, user_message, example_message, feedback_message
from utils import save_solution_puml, save_chat_json, save_chat_text

In [2]:
SEED = 1234

base_path = Path().cwd()
cpp_path = base_path / "dishwasher_cpp/host"

diagrams_path = base_path / "diagrams"
state_diagram_path = diagrams_path / "state_diagram"

chat_path = base_path / "chats" / "domain_examples"
chat_path.mkdir(parents=True, exist_ok=True)

xml_path = base_path / "xml_data"

load_dotenv(override=True)

True

In [8]:
# Generate the example prompts for each core components
core_components_file = base_path / "core_components_openai.json"
core_components_list = json.loads(core_components_file.read_text())


components_files = {}


for target_component, code_files_raw in core_components_list.items():

    # get code files for the target component
    code_files = []
    for c in code_files_raw:
        code_files.append(base_path / c.replace("\\", "/").replace(".h", ".cpp"))

    image_files = list(
        (state_diagram_path / f"{target_component}/ground_truth").glob("*.jpg")
    )

    if len(code_files) > 0:
        components_files[target_component] = [code_files, image_files[0] if len(image_files) > 0 else None]

In [4]:
# format chatgpt response format
class Solution(BaseModel):
    solution: str = Field(
        description="PlantUML solution that start with '@startuml' and end with '@enduml'"
    )

In [10]:
for target_component, code_files_raw in core_components_list.items():

    temp = components_files.pop(target_component)

    # if cpp files do not exist, skip the component
    if all(not c.is_file() for c in temp[0]):
        print(f"No code files found for component: {target_component}")
        continue

    target_component = target_component.lower()
    print(f"Processing component: {target_component}")

    # User Prompt containing target cpp
    user_prompt = user_message(temp[0])

    # collect domain examples for other components
    examples = [
        example_message(v[0], v[1])
        for c, v in components_files.items()
        if v[1] is not None
    ]
    # flatten the examples list
    examples = [e2 for e in examples for e2 in e]

    # readd the target component to the components_files
    components_files[target_component] = temp

    # System Prompt
    messages = SYSTEM_PROMPT + examples + user_prompt

    client = AzureOpenAI(
        api_key=os.getenv("AZURE_API_KEY"),
        api_version=os.getenv("AZURE_API_VERSION"),
        azure_endpoint=os.getenv("AZURE_API_BASE"),
    )

    response = client.beta.chat.completions.parse(
        model=os.getenv("MODEL"),
        temperature=0.2,
        messages=messages,
        seed=SEED,
        response_format=Solution,
    )

    # parse output
    solution = json.loads(response.choices[0].message.content)["solution"]

    # save solution to puml file, render it, and save messages
    chat_path_component = chat_path / target_component
    chat_path_component.mkdir(parents=True, exist_ok=True)

    chat_filename = f"state_general_{target_component}_{response.model}"
    date_postfix = datetime.datetime.now().strftime("%d-%m-%Y_%H-%M-%S")

    save_solution_puml(
        solution, state_diagram_path, target_component, response.model, date_postfix
    )
    save_chat_json(
        messages, solution, response, chat_path_component, chat_filename, date_postfix
    )
    save_chat_text(messages, chat_path_component, chat_filename, date_postfix)

Processing component: abstractfactory
Processing component: dishwasher
Processing component: heater
Processing component: jet
Processing component: tank
